<a href="https://colab.research.google.com/github/Edu963/ichimoku/blob/main/Ichimoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
# Download historical stock price data
df = yf.download("HUBS", start="2024-03-11", end="2024-12-31", interval='15m')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
def ichimoku_cloud(df, n_delay=26, k_delay=9, lag_span=52):
    tenkan_sen_high = df['High'].rolling(window=n_delay).max()
    tenkan_sen_low = df['Low'].rolling(window=n_delay).min()
    tenkan_sen = (tenkan_sen_high + tenkan_sen_low) / 2
    kijun_sen_high = df['High'].rolling(window=k_delay).max()
    kijun_sen_low = df['Low'].rolling(window=k_delay).min()
    kijun_sen = (kijun_sen_high + kijun_sen_low) / 2
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(k_delay)
    lagging_span_high = df['High'].rolling(window=lag_span).max()
    lagging_span_low = df['Low'].rolling(window=lag_span).min()
    senkou_span_b = ((lagging_span_high + lagging_span_low) / 2).shift(lag_span)
    chikou_span = df['Close'].shift(-k_delay)
    return pd.DataFrame({
        'Tenkan-Sen': tenkan_sen,
        'Kijun-Sen': kijun_sen,
        'Senkou Span A': senkou_span_a,
        'Senkou Span B': senkou_span_b,
        'Chikou Span': chikou_span,
        'Close': df['Close'],
        'Open': df['Open'],
        'High': df['High'],
        'Low': df['Low']
    }, index=df.index)
ichimoku = ichimoku_cloud(df)

print("Tenkan-Sen:\n", ichimoku['Tenkan-Sen'].head(10))
print("\nKijun-Sen:\n", ichimoku['Kijun-Sen'].head(10))
print("\nSenkou Span A:\n", ichimoku['Senkou Span A'].head(10))
print("\nSenkou Span B:\n", ichimoku['Senkou Span B'].head(10))
print("\nChikou Span:\n", ichimoku['Chikou Span'].head(10))

    #return ichimoku


Tenkan-Sen:
 Datetime
2024-03-11 09:30:00   NaN
2024-03-11 09:45:00   NaN
2024-03-11 10:00:00   NaN
2024-03-11 10:15:00   NaN
2024-03-11 10:30:00   NaN
2024-03-11 10:45:00   NaN
2024-03-11 11:00:00   NaN
2024-03-11 11:15:00   NaN
2024-03-11 11:30:00   NaN
2024-03-11 11:45:00   NaN
Name: Tenkan-Sen, dtype: float64

Kijun-Sen:
 Datetime
2024-03-11 09:30:00           NaN
2024-03-11 09:45:00           NaN
2024-03-11 10:00:00           NaN
2024-03-11 10:15:00           NaN
2024-03-11 10:30:00           NaN
2024-03-11 10:45:00           NaN
2024-03-11 11:00:00           NaN
2024-03-11 11:15:00           NaN
2024-03-11 11:30:00    608.024994
2024-03-11 11:45:00    608.024994
Name: Kijun-Sen, dtype: float64

Senkou Span A:
 Datetime
2024-03-11 09:30:00   NaN
2024-03-11 09:45:00   NaN
2024-03-11 10:00:00   NaN
2024-03-11 10:15:00   NaN
2024-03-11 10:30:00   NaN
2024-03-11 10:45:00   NaN
2024-03-11 11:00:00   NaN
2024-03-11 11:15:00   NaN
2024-03-11 11:30:00   NaN
2024-03-11 11:45:00   NaN
Name:

In [ ]:
def ichimoku_trading_signals(df, ichimoku, displacement=26):
    buy_signal = (
        (df['Close'].shift(-displacement) > ichimoku['Senkou Span B']) &
        (np.minimum(df['Close'], df['Open']) > ichimoku['Kijun-Sen']) &
        (df['Close'].shift(-displacement) > ichimoku['Senkou Span B'].shift(-displacement))
    )
    sell_signal = (
        (df['Close'].shift(-displacement) < ichimoku['Senkou Span B']) &
        (np.minimum(df['Close'], df['Open']) < ichimoku['Kijun-Sen']) &
        (df['Close'].shift(-displacement) < ichimoku['Senkou Span B'].shift(-displacement))
    )
    signals = pd.DataFrame({
        'Buy Signal': buy_signal,
        'Sell Signal': sell_signal
    }, index=df.index)
    return signals





In [ ]:
# Calculate Ichimoku Cloud
ichimoku = ichimoku_cloud(df)

# Generate trading signals
signals = ichimoku_trading_signals(df, ichimoku, displacement=26)

In [ ]:
print("Buy signals:", signals['Buy Signal'].sum())
print("Sell signals:", signals['Sell Signal'].sum())

Buy signals: 167
Sell signals: 272


In [ ]:
def backtest(df, signals):
    df['Position'] = np.where(signals['Buy Signal'], 1, 0)
    df['Position'] = np.where(signals['Sell Signal'], -1, df['Position'])
    df['Position'] = df['Position'].ffill().fillna(0)
    df['PnL'] = df['Position'].shift(1) * df['Close'].pct_change()
    df['Cumulative PnL'] = df['PnL'].cumsum()
    return df['Cumulative PnL']

cumulative_pnl = backtest(df, signals)
print(cumulative_pnl.tail())

Datetime
2024-05-02 11:15:00    0.216557
2024-05-02 11:30:00    0.216557
2024-05-02 11:45:00    0.216557
2024-05-02 12:00:00    0.216557
2024-05-02 12:15:00    0.216557
Name: Cumulative PnL, dtype: float64


In [ ]:
def evaluate_performance(df, signals):
    # Calculate strategy returns
    df['Position'] = np.where(signals['Buy Signal'], 1, np.where(signals['Sell Signal'], -1, 0))
    df['Position'] = df['Position'].ffill().fillna(0)
    df['Returns'] = df['Close'].pct_change()
    df['Strategy Returns'] = df['Position'].shift(1) * df['Returns']

    # Calculate cumulative returns for the strategy
    df['Cumulative Strategy Returns'] = (1 + df['Strategy Returns']).cumprod() - 1

    # Calculate performance metrics
    total_return = df['Cumulative Strategy Returns'].iloc[-1]
    max_drawdown = (df['Cumulative Strategy Returns'].cummax() - df['Cumulative Strategy Returns']).max()
    annualized_sharpe_ratio = (df['Strategy Returns'].mean() / df['Strategy Returns'].std()) * np.sqrt(252)

    trades = signals['Buy Signal'].sum() + signals['Sell Signal'].sum()
    average_profit_loss_per_trade = df['Strategy Returns'].sum() / trades if trades != 0 else 0

    performance_metrics = {
        'Total Return': f'{total_return * 100:.2f}%',
        'Max Drawdown': f'{max_drawdown * 100:.2f}%',
        'Annualized Sharpe Ratio': f'{annualized_sharpe_ratio:.2f}',
        'Number of Trades': trades,
        'Average Profit/Loss per Trade': f'{average_profit_loss_per_trade * 100:.2f}%'
    }

    return performance_metrics



In [ ]:
performance_metrics = evaluate_performance(df, signals)
print(performance_metrics)

{'Total Return': '23.62%', 'Max Drawdown': '4.27%', 'Annualized Sharpe Ratio': '1.16', 'Number of Trades': 439, 'Average Profit/Loss per Trade': '0.05%'}


In [ ]:
# Setup for grid search
n_delays = range(9, 37, 3)
k_delays = range(3, 19, 3)
lag_spans = range(26, 79, 13)

results = []

for n in n_delays:
    for k in k_delays:
        for lag in lag_spans:
            ichimoku = ichimoku_cloud(df, n_delay=n, k_delay=k, lag_span=lag)
            signals = ichimoku_trading_signals(df, ichimoku)
            performance_metrics = evaluate_performance(df, signals)
            results.append((n, k, lag, performance_metrics))

# Analyze results to find the best parameters
best_result = sorted(results, key=lambda x: x[3]['Total Return'], reverse=True)[0]
print("Best parameters:", best_result)

Best parameters: (9, 3, 78, {'Total Return': '9.63%', 'Max Drawdown': '4.88%', 'Annualized Sharpe Ratio': '0.52', 'Number of Trades': 380, 'Average Profit/Loss per Trade': '0.03%'})
